# Trabajo Práctico 2

## Consigna

En este práctico se trabajará con el dataset de Rossmann. Rossmann es una cadena de drugstores con más de 3000 tiendas en 7 países de Europa. La tarea que se intentará resolver con este dataset es el de predicción de las ventas totales de cada tienda que tendrán lugar en un intervalo de 6 semanas. Esta competencia fue presentada en Kaggle y se puede acceder a sus recursos relacionados:

https://www.kaggle.com/c/rossmann-store-sales

Este práctico consiste en generar un modelo que realice la inferencia para el problema planteado. Los entregables consisten en:

Compartir el link de un repositorio que contenga:

- La/s notebook/s con la resolución del ejercicio y todo el código propio utilizado para correr la/s notebooks.
- Un archivo requirements.txt con las versiones de las librerías de Python utilizadas (en formato pip)
- Captura de pantalla con el máximo score obtenido al hacer el late submission y el submission.csv utilizado.

Además, la notebook principal debe incluir lo siguiente:

- Una explicación detallada de cómo se preprocesaron los datos (aquellos provistos para la competencia y datos externos)
- Una tabla con los distintos modelos e hiperparámetros que se hayan probado y sus respectivos scores.
- Gráficos con los embeddings obtenidos.
- Las respuestas a las siguientes preguntas:
  - ¿Qué son los Entity Embeddings y cómo se relacionan con las variables categóricas?
  - Explique la métrica utilizada en la competencia.

## Datasets

Los datos utilizados fueron los provistos para la competencia (train, test y store) y los externos utilizados por los que ganaron el 3er puesto de la competencia.


### Datasets provistos por la competencia

$\textbf{Kaggle Challenge de Rossmann Store Sales}$

https://www.kaggle.com/competitions/rossmann-store-sales/overview


$\textbf{Files:}$
* train.csv - historical data including Sales
* test.csv - historical data excluding Sales
* sample_submission.csv - a sample submission file in the correct format
* store.csv - supplemental information about the stores


$\textbf{Data fields:}$
Most of the fields are self-explanatory. The following are descriptions for those that aren't.

* Id - an Id that represents a (Store, Date) duple within the test set
* Store - a unique Id for each store
* Sales - the turnover for any given day (this is what you are predicting)
* Customers - the number of customers on a given day
* Open - an indicator for whether the store was open: 0 = closed, 1 = open
* StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
* SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
* StoreType - differentiates between 4 different store models: a, b, c, d
Assortment - describes an assortment level: a = basic, b = extra, c = extended
* CompetitionDistance - distance in meters to the nearest competitor store
* CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
* Promo - indicates whether a store is running a promo on that day
* Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
* PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store


### Datasets externos

Los datasets externos utilizados fueron los siguientes:

* store_states.csv - indica el estado en el que se encuentra cada una de las tiendas
* state_names.csv - indica el acrónimo de cada estado
* googletrend.csv - indica las tendencias por semana
* wheather.csv - indica las condiciones meteorológicas por día

## Preprocesamiento de datos

Los datos utilizados se preprocesan el los notebooks '02-data-preprocess.ipynb', '03-durations-preprocess.ipynb' y '04-Normalize-and_encode.ipynb'.

$\textbf{En el Notebook '02-data-preprocess.ipynb':}$

El preprocesamiento se inició revisando los datasets en busca de datos faltantes. Se encontraron datos faltantes en variables de los datasets test, store y weather, y estos fueron tratados de distintas formas:
* En test, los datos faltantes de la variable 'Open' se reemplazaron por 1 (abierto) a menos de que fuese día 7 (domingo) ya que usualmente las tiendas se encuentran cerradas.
* En store, los datos faltantes de las variables 'CompetitionOpenSinceMonth' y 'CompetitionOpenSinceYear' (o los anteriores al año 1990) fueron reemplazados por el mes 1 (enero) de 1990.
* En store, los datos faltantes de la variable 'CompetitionDistance' se consideraron como lejanos y fueron reemplazados por el valor máximo de la variable (75860).
* En store, los datos faltantes de las variables 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval' que no tenian Promo2 fueron reemplazados por la primera semana de 1990 y duración nula (-)
* En weather, los datos faltantes de las variables 'Max_VisibilityKm', 'Mean_VisibilityKm', 'Min_VisibilitykM', 'Max_Gust_SpeedKm_h', 'CloudCover' y 'Events' fueron reemplazados por condiciones climáticas tranquilas, utilizando los máximos y mínimos de estas variables.

Luego, en el dataset store se crearon nuevas columnas con las fechas en formato año-mes-día para minimizar el número de variables.
En el dataset googletrend, se creó una variable con el acrónimo del estado en el que se encuentra la tienda a partir de un split de la variable 'File' del mismo.

Después, se unieron los datasets para obtener un nuevo train y test, juntandolos en el siguiente orden:
* weather y state_names en weather
* store y store_states en store
* train y states en joined_train
* test y states en joined_test
* joined_train y googletrend en joined_train
* joined_test y googletrend en joined_test
* joined_train y weather en joined_train
* joined_test y weather en joined_test

Para continuar se agregaron 2 columnas llamadas 'CompetitionDaysOpen' y 'Promo2Days' obtenidas a partir de las variables 'CompetitionOpenSinceMonth' y 'CompetitionOpenSinceYear', y 'Promo2SinceWeek' y 'Promo2SinceYear', respectivamente. Estas nuevas variables se corrigieron para estar en meses, no tener valores negativos y tener un máximo de 24 meses (2 años).

Por último se eliminan los datos duplicados, y se almacenan los nuevos datasets train y test en archivos .fth.




$\textbf{En el Notebook '03-durations-preprocess.ipynb':}$

Acá se generaron nuevas columnas del tiempo before (antes) y after (después) de eventos (StateHoliday, SchoolHoliday, Promo). Luego se eliminaron los datos NaT (Not a Time), y se suavizaron las columnas de valores temporales. Por úlimo, se almacenaron una vez más los datasets train y test en archivos .fth.



$\textbf{En el Notebook '04-Normalize-and_encode.ipynb':}$

Aquí se separan las variables en categóricas y continuas. Las variables categóricas se codificaron usando la función LabelEncoder, y las continuas se estandarizaron usando la función StandardScaler, ambas de la librería sklearn.

También se eliminaron los datos en los que las ventas eran nulas. Finalmente, se almacenaron nuevamente los datasets train y test en archivos .fth.



## Modelos e hiperparámetros

Se 'submitearon' varios modelos generados en las notebooks '06-full-model.ipynb', '07-lightGBM.ipynb', '08-XGBoost.ipynb' y '09-lightGBM-hyp-search.ipynb'. En la siguiente tabla se muestran los modelos enviados, algunos de sus parámetros y el score que se obtuvo.

| Notebook | Modelo | Learning rate | Epochs | Batch size | Min child Samples | Score |
|--------|--------|---------------|--------|------------|-------------------|-------|
|06-full-model|Red neuronal con embeddings |0.001|20|256|-|0.25305|
|08-XGBoost|XGBRegressor             |0.25 |-|-|-|0.22672|
|09-lightGBM-hyp-search|LGBMRegressor con Hyp Search|0.05|-|-|5|0.13695|
|07-lightGBM|LGBMRegressor           |0.05 |-|-|5|0.13564|


$\textbf{Mejor score obtenido}$:

En la siguiente figura se muestra la captura de pantalla del mejor score obtenido, generado con el modelo LGBMRegressor ubicado en la notebook '07-lightGBM.ipynb'.

![picture](https://drive.google.com/uc?export=view&id=1bxgmfn_Ae2FUulGeynmU1dD50qiGkQiw)


## Embeddings

Los embeddings obtenidos fueron los de las variables 'DayOfWeek' y 'Store'.

Los embeddings de la variable 'DayOfWeek' se muestran en la siguiente figura:

![picture](https://drive.google.com/uc?export=view&id=1JBrU80gSlbqSpPUhzjXSZ9m07YKA1irk)

En la imagen anterior se puede notar que los días entre semana (Monday-Friday) se encuentran más cercanos entre sí, y los días de fin de semana (Saturday y Sunday) lejos de los anteriores, lo cual hace sentido en cuanto al comportamiento de las ventas en general. En particular, se puede notar una alta proximidad entre los días Wednesday y Thursday.

Los embeddings de la variable 'Store' se muestran en la siguiente figura:

![picture](https://drive.google.com/uc?export=view&id=1GmVZWMeAaXi5UIRVXSEW8Sr6CXmH1-j-)

En esta imagen se puede ver que las tiendas forman un cluster, y que las tiendas 84, 787 y sobre todo la 816 son las más aisladas.

## Preguntas

### ¿Qué son los Entity Embeddings y cómo se relacionan con las variables categóricas?

En esencia, los entity embeddings son vectores que representan algo, una entidad. Las variables categóricas pueden ser representadas de forma compacta y continua mediante el mapeo realizado mediante entity embedding.

De esta forma, se puede pasar de variables categóricas a continuas, siendo estas de preferencia a la hora de aplicar redes neuronales. Los valores de una variable categórica siempre tienen algún tipo de relación intrínseca. Otra alternativa para aplicar redes neuronales a problemas con variables categóricas es one-hot-encoding, pero este método ignora las relaciones entre los valores de una variable. Por el contrario, entity embedding revela las propiedades intrínsecas de las variables categóricas mediante el mapeo cercano de valores similares. Además, entity embedding reduce el uso de memoria y acelera la performance de las redes neuronales.


### Explique la métrica utilizada en la competencia

La métrica utilizada en esta competencia es la raíz cuadrada del error cuadrático medio porcentual (RMSPE). El mismo se calcula de la siguiente forma:

$RMSPE = \sqrt{\frac{1}{n} \sum_{i=1}^{n}(\frac{y_i - \hat{y_i}}{y_i})^2}$

A partir de la fórmula vemos que el RMSPE realiza una suma de los errores de todos los datos. Este error tiene en cuenta la diferencia entre los valor predichos y los reales. Se utiliza el valor cuadrático, por lo que se ignora el signo de estas diferencias. A su vez, se divide por el valor real para obtenerlo en forma de porcentaje, lo que hace que el tamaño de este error no dependa del orden de magnitud de la variable y. Por esto, el RMSPE resulta una métrica bastante objetiva y apropiada para esta competencia.

